In [1]:
from PIL import Image
import numpy as np
import cv2
from scipy import ndimage
import matplotlib.pyplot as plt

In [2]:
def rotate(image_path, degrees_to_rotate):
    """
    Rotate the given photo the amount of given degreesk, show it and save it
    @param image_path: The path to the image to edit
    @param degrees_to_rotate: The number of degrees to rotate the image
    @param saved_location: Path to save the cropped image
    """
    image_obj = Image.open(image_path)
    rotated_image = image_obj.rotate(degrees_to_rotate)
    rotated_image.save(image_path)

if __name__ == '__main__':
    #image = 'out15.jpg'
    #rotate(image, 90)

SyntaxError: unexpected EOF while parsing (<ipython-input-2-7990bf45f4ba>, line 14)

In [4]:
def grab_cut(img, full_scr=True, iters=3, rect=(50, 50, 450, 290)):
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    if full_scr:
        rect = (2, 2, img.shape[1] - 2, img.shape[0] - 2)
        cv2.grabCut(img, mask, rect, bgdModel, fgdModel, iters, cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        img = img * mask2[:, :, np.newaxis]
    return img


def find_doc_by_grab_cut(orig_image, iters=3, border=50):
    image = orig_image.copy()
    ratio = orig_image.shape[0] / 500.0

    mask = np.zeros(image.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (2, 2, image.shape[1] - 2, image.shape[0] - 2)
    cv2.grabCut(image, mask, rect, bgdModel, fgdModel, iters, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    mask2 = mask2.reshape(image.shape[0], image.shape[1]) * 255.
    mask2 = cv2.copyMakeBorder(mask2.astype('uint8'), border, border, border, border, cv2.BORDER_CONSTANT)

    contours = cv2.findContours(mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    main_contours = find_rects(contours, 500 * 500) * ratio - border
    if len(main_contours) > 0:
        main_contour = main_contours[0]

        points = np.asarray(main_contour).reshape(4, 2)
        document = four_point_transform(orig_image.copy(), points)
        return document
    else:
        return orig_image

In [5]:
image = plt.imread('out16.jpg')

In [ ]:
image = find_doc_by_grab_cut(image)